In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

# RID OSPAR Reporting Template

**Note:** Despite the [changes to the programme](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/programme_changes_2017-18.ipynb) for 2017 to 2020, we have decided to continue to submit data to OSPAR using the old divisions (see e-mail from Øyvind received 15.10.2018 at 15.51). The code in this notebook therefore uses the column named `'old_rid_code'` to allocate fluxes to the correct regions.

The final step in the annual RID reporting is to complete and submit an overall summary template to OSPAR. A blank template and guideline documentation were provided by Eva (see e-mail received 04/09/2017 at 13.07) and the 2015 template created by Tore is here:

K:\Avdeling\Vass\316_Miljøinformatikk\Prosjekter\RID\2016\OSPAR\Norway_2015.xlsx

**Note:** Tore previous calculated "upper" load estimates by setting LOD values equal to the LOD itself, and "lower" estimates by setting LOD values equal to zero. This year we have handled LOD values more explicity, by calculating a single estimate as described [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/rid_data_exploration.ipynb#2.3.-Calculate-loads). We therefore only need to fill-in entires for the "mean" rows.

Most of the information required for the template has already been reported in other tables, especially the following:

 * *Table_3_2016.docx* <br><br>
 
 * *loads_and_flows_all_sites_2016.csv* <br><br>
 
 * *concs_and_flows_rid_11-36_2016.csv*

However, some information is not available from these tables, and in previous years Tore has simply left these cells blank. The code here does the same, although in principle we could fill-in the whole thing with a bit more work.

## 1. Get summary data

The notebook [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/summary_table_2016.ipynb) calculates summary values for Table 3. Rather than reading these values from Word, it is easier to repeat the code to summarise the raw data again. This is the data that needs writing to the OSPAR template.

### 1.1. Monitored areas

In [2]:
# Read data
in_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
          r'\Results\Loads_CSVs\loads_and_flows_all_sites_2017.csv')
mon_df = pd.read_csv(in_csv)
del mon_df['new_rid_group']

# Group by OSPAR region
mon_df1 = mon_df.groupby(['ospar_region', 'old_rid_group']).sum()

# Totals for Norway
mon_df2 = mon_df.groupby('old_rid_group').sum().reset_index()
mon_df2['ospar_region'] = 'NORWAY'
mon_df2.set_index(['ospar_region', 'old_rid_group'], inplace=True)

# Combine
mon_df = pd.concat([mon_df1, mon_df2], axis=0)

# Cols of interest
cols = [i for i in mon_df.columns if i.split('_')[1] != 'Est']
mon_df = mon_df[cols]
del mon_df['station_id'], mon_df['mean_q_1000m3/day']

# Convert units
mon_df['Hg_kg'] = mon_df['Hg_kg']/1000. # kg to tonnes
mon_df['NH4-N_tonnes'] = mon_df['NH4-N_tonnes']/1000. # tonnes to ktonnes
mon_df['NO3-N_tonnes'] = mon_df['NO3-N_tonnes']/1000. # tonnes to ktonnes
mon_df['TOTN_tonnes'] = mon_df['TOTN_tonnes']/1000. # tonnes to ktonnes
mon_df['TOTP_tonnes'] = mon_df['TOTP_tonnes']/1000. # tonnes to ktonnes
mon_df['PO4-P_tonnes'] = mon_df['PO4-P_tonnes']/1000. # tonnes to ktonnes
mon_df['SPM_tonnes'] = mon_df['SPM_tonnes']/1000. # tonnes to ktonnes

# Units are correct, so remove
mon_df.columns = [i.split('_')[0] for i in mon_df.columns]

mon_df.round(0)

Ag    As   Cd    Cr    Cu   Hg  NH4-N  \
ospar_region        old_rid_group                                           
LOFOTEN-BARENTS SEA rid_108        0.0   1.0  0.0   1.0   7.0  0.0    0.0   
                    rid_11         0.0   0.0  0.0   1.0   2.0  0.0    0.0   
                    rid_36         0.0   2.0  0.0   5.0  27.0  0.0    0.0   
NORTH SEA           rid_108        0.0   2.0  0.0   1.0  11.0  0.0    0.0   
                    rid_11         0.0   0.0  0.0   0.0   1.0  0.0    0.0   
                    rid_36         0.0   2.0  0.0   3.0  12.0  0.0    0.0   
NORWEGIAN SEA2      rid_108        0.0   1.0  0.0   1.0   7.0  0.0    0.0   
                    rid_11         0.0   1.0  0.0   1.0  13.0  0.0    0.0   
                    rid_36         0.0   3.0  0.0   8.0  25.0  0.0    0.0   
SKAGERAK            rid_108        0.0   1.0  0.0   1.0   3.0  0.0    0.0   
                    rid_11         0.0   8.0  1.0   9.0  50.0  0.0    1.0   
                    rid_36         0.0   2.0  0.0   1.0   7.0  0.0    0.0   
NORWAY              rid_108        0.0   5.0  0.0   4.0  29.0  0.0    1.0   
                    rid_11         0.0  10.0  1.0  11.0  66.0  0.0    1.0   
                    rid_36         0.0   9.0  1.0  16.0  71.0  0.0    1.0   

                                   NO3-N    Ni  PO4-P    Pb    SPM      SiO2  \
ospar_region        old_rid_group                                              
LOFOTEN-BARENTS SEA rid_108          0.0   6.0    0.0   0.0   12.0   28725.0   
                    rid_11           0.0   1.0    0.0   0.0    7.0   16367.0   
                    rid_36           1.0  56.0    0.0   1.0   61.0   92473.0   
NORTH SEA           rid_108          4.0   8.0    0.0   4.0   28.0   31558.0   
                    rid_11           0.0   1.0    0.0   0.0    5.0    3989.0   
                    rid_36           4.0   4.0    0.0   4.0   42.0   38036.0   
NORWEGIAN SEA2      rid_108          1.0   6.0    0.0   1.0   31.0   25375.0   
                    rid_11           0.0   3.0    0.0   0.0    8.0   10405.0   
                    rid_36           3.0  21.0    0.0   1.0  100.0   72013.0   
SKAGERAK            rid_108          1.0   2.0    0.0   1.0   13.0    9773.0   
                    rid_11          14.0  28.0    0.0  10.0  264.0  163721.0   
                    rid_36           1.0   3.0    0.0   3.0   16.0   21453.0   
NORWAY              rid_108          6.0  21.0    0.0   6.0   84.0   95431.0   
                    rid_11          15.0  33.0    0.0  10.0  283.0  194483.0   
                    rid_36           9.0  84.0    0.0  10.0  219.0  223976.0   

                                        TOC  TOTN  TOTP     Zn  
ospar_region        old_rid_group                               
LOFOTEN-BARENTS SEA rid_108         30846.0   1.0   0.0    8.0  
                    rid_11          12030.0   1.0   0.0    1.0  
                    rid_36          71082.0   4.0   0.0   27.0  
NORTH SEA           rid_108         32615.0   6.0   0.0   34.0  
                    rid_11           5335.0   1.0   0.0    4.0  
                    rid_36          48394.0   8.0   0.0   57.0  
NORWEGIAN SEA2      rid_108         27538.0   2.0   0.0   12.0  
                    rid_11          11981.0   1.0   0.0   21.0  
                    rid_36          69705.0   7.0   0.0   58.0  
SKAGERAK            rid_108         16711.0   2.0   0.0   22.0  
                    rid_11         208394.0  23.0   1.0  124.0  
                    rid_36          51078.0   4.0   0.0   39.0  
NORWAY              rid_108        107709.0  12.0   0.0   75.0  
                    rid_11         237740.0  25.0   1.0  150.0  
                    rid_36         240258.0  22.0   1.0  181.0

### 1.2. Unmonitored areas

In [3]:
# Read data
in_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
          r'\Results\Unmon_loads\unmon_loads_2017.csv')
umon_df = pd.read_csv(in_csv, index_col=0)

# Rename cols
umon_df.columns = [i.replace('RENSEANLEGG', 'sew') for i in umon_df.columns]
umon_df.columns = [i.replace('INDUSTRI', 'ind') for i in umon_df.columns]
umon_df.columns = [i.replace('_tonn', '') for i in umon_df.columns]
umon_df.columns = [i.replace('AQUAKULTUR', 'fish') for i in umon_df.columns]

# Convert Hg to kgs
umon_df['sew_Hg'] = umon_df['sew_Hg']*1000
umon_df['ind_Hg'] = umon_df['ind_Hg']*1000

umon_df.round(0)

,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p,sew_po4,...,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg,fish_Cu
ospar_region,,,,,,,,,,,,,,,,,,,,,
NORWAY,507481.0,13971.0,1074.0,2471.0,290.0,59962.0,10350.0,39601.0,771.0,645.0,...,13637.0,0.0,0.0,0.0,5.0,20.0,3.0,1.0,10.0,1088.0
LOFOTEN-BARENTS SEA,138847.0,1205.0,163.0,81.0,6.0,15814.0,2731.0,6359.0,125.0,98.0,...,2357.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,286.0
NORTH SEA,174898.0,3735.0,432.0,529.0,77.0,19137.0,3299.0,15986.0,253.0,259.0,...,5429.0,0.0,0.0,0.0,2.0,9.0,1.0,1.0,4.0,346.0
NORWEGIAN SEA2,183121.0,2735.0,351.0,911.0,158.0,24982.0,4314.0,14197.0,305.0,211.0,...,3531.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,3.0,455.0
SKAGERAK,10615.0,6296.0,128.0,951.0,50.0,29.0,5.0,3059.0,88.0,77.0,...,2319.0,0.0,0.0,0.0,2.0,9.0,1.0,0.0,2.0,1.0


### 1.3. Loads for 11 main rivers

In [4]:
# Read data
in_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
          r'\Results\Loads_CSVs\loads_and_flows_all_sites_2017.csv')
rid11_df = pd.read_csv(in_csv, index_col=0)
del rid11_df['new_rid_group']

# Get data for RID11
rid11_df = rid11_df.query('old_rid_group == "rid_11"')

# Tidy
del rid11_df['station_code'], rid11_df['station_name']
del rid11_df['old_rid_group'], rid11_df['ospar_region']
del rid11_df['mean_q_1000m3/day']

cols = [i for i in rid11_df.columns if i.split('_')[1] != 'Est']
rid11_df = rid11_df[cols]

# Convert units
rid11_df['Hg_kg'] = rid11_df['Hg_kg']/1000. # kg to tonnes
rid11_df['NH4-N_tonnes'] = rid11_df['NH4-N_tonnes']/1000. # tonnes to ktonnes
rid11_df['NO3-N_tonnes'] = rid11_df['NO3-N_tonnes']/1000. # tonnes to ktonnes
rid11_df['TOTN_tonnes'] = rid11_df['TOTN_tonnes']/1000. # tonnes to ktonnes
rid11_df['TOTP_tonnes'] = rid11_df['TOTP_tonnes']/1000. # tonnes to ktonnes
rid11_df['PO4-P_tonnes'] = rid11_df['PO4-P_tonnes']/1000. # tonnes to ktonnes
rid11_df['SPM_tonnes'] = rid11_df['SPM_tonnes']/1000. # tonnes to ktonnes

# Tidy cols
rid11_df.columns = [i.split('_')[0] for i in rid11_df.columns]

rid11_df.head()

,Ag,As,Cd,Cr,Cu,Hg,NH4-N,NO3-N,Ni,PO4-P,Pb,SPM,SiO2,TOC,TOTN,TOTP,Zn
station_id,,,,,,,,,,,,,,,,,
29612,0.000000,1.885934,0.091354,1.704438,8.229109,0.012560,0.103427,2.618654,5.265423,0.026224,1.444168,20.771381,31359.804331,41294.356441,4.256075,0.067885,24.325649
29779,0.000000,0.413012,0.000000,0.682191,1.719991,0.003129,0.012023,0.194927,0.870596,0.015467,0.044949,6.845721,16367.010804,12029.950310,0.704780,0.026165,0.576850
29821,0.000000,0.207549,0.013841,0.162546,1.044845,0.000000,0.015083,0.307519,0.815021,0.003429,0.158681,1.942288,3063.193676,3805.132158,0.500895,0.009256,2.829575
29782,0.000000,0.359566,0.009270,0.381896,1.064020,0.000000,0.003611,0.094011,0.982699,0.003345,0.191518,4.889757,4305.804503,5793.362515,0.327468,0.011089,1.047347
36225,0.000975,0.026755,0.005583,0.095556,0.426425,0.000067,0.003227,0.036341,0.113191,0.002043,0.161677,0.433375,290.341819,200.175307,0.060726,0.002603,1.853462


## 2. Fill-in template

A copy of the 2017 template from Eva is here:

C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet\Results\OSPAR_Template\01_OSPAR_Norway_2016.xlsx

In [5]:
# Copy of template to update
temp_path = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
             r'\Results\OSPAR_Template\01_OSPAR_Norway_2017.xlsx')

In [6]:
def update_spreadsheet_point_sources(xlsx, sheet, pars, src, df):
    """ Update the OSPAR template for point source data.
    
    Args:
        xslx:  Str. Path to Excel template
        sheet: Str. Sheet name to update
        pars:  List. Parameter names in template to fill-in
        src:   Str. Type of input ('sew', 'ind', 'fish')
        df:    Dataframe. Values to fill-in
        
    Returns:
        None. The template is updated and saved.
    """
    import pandas as pd
    from openpyxl import load_workbook
    
    # Map Excel headings to df cols
    par_dict = {'SPM':'S.P.M.',
                'TOC':'TOC',
                'PO4-P':'po4',
                'P-Total':'p',
                'NO3-N':'no3', 
                'NH4-N':'nh4',
                'N-Total':'n',
                'As':'As',
                'Pb':'Pb',
                'Cd':'Cd', 
                'Cu':'Cu',
                'Zn':'Zn',
                'Ni':'Ni',
                'Total Cr':'Cr',
                'Hg':'Hg'}

    # Map template names to df names
    names_dict = {'Norwegian Sea (NO)':'NORWEGIAN SEA2',
                  'Barents Sea (NO)':'LOFOTEN-BARENTS SEA',
                  'Skagerrak (NO)':'SKAGERAK',
                  'North Sea (NO)':'NORTH SEA',
                  'Norway Total':'NORWAY'}

    # Open new file and get sheet
    wb = load_workbook(filename=xlsx)
    ws = wb[sheet]

    # Get row numbers
    row_dict = {}
    for item in ws['B12':'B%s' % ws.max_row]:
        # Get cell properties
        cell = item[0]
        name = cell.value
        row = cell.row
        row_dict[name] = row

    # Get col numbers
    col_dict = {}
    for cell in ws['E9':'AK9'][0]:
        # Get cell properties
        par = cell.value
        col = cell.column
        col_dict[par] = col

    # Update spreadsheet
    for reg in names_dict.keys():
        for par in pars:
            # Get value from df
            val = df.ix[names_dict[reg], '%s_%s' % (src, par_dict[par])]

            # Get cell co-ords
            row = row_dict[reg] + 2
            col = col_dict[par]

            # Write value
            ws['%s%s' % (col, row)] = val

    # Save
    wb.save(xlsx)

### 2.1. Sheet 5a: Sewage effluents

Note these are the values from Table 3 for **unmonitored areas** and not the total sewage inputs for each of the OSPAR areas. This is the same as what Tore reported previously.

In [7]:
# Get Sewage data
cols = [i for i in umon_df.columns if i.split('_')[0]=='sew']
sew_df = umon_df[cols].copy()

# Convert units
sew_df['sew_Hg'] = sew_df['sew_Hg']/1000. # kg to tonnes
sew_df['sew_nh4'] = sew_df['sew_nh4']/1000. # tonnes to ktonnes
sew_df['sew_no3'] = sew_df['sew_no3']/1000. # tonnes to ktonnes
sew_df['sew_n'] = sew_df['sew_n']/1000. # tonnes to ktonnes
sew_df['sew_po4'] = sew_df['sew_po4']/1000. # tonnes to ktonnes
sew_df['sew_p'] = sew_df['sew_p']/1000. # tonnes to ktonnes
sew_df['sew_S.P.M.'] = sew_df['sew_S.P.M.']/1000. # tonnes to ktonnes

sew_df

,sew_n,sew_p,sew_po4,sew_no3,sew_nh4,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg
ospar_region,,,,,,,,,,,,,,
NORWAY,13.970939,1.074235,0.644541,0.698547,10.478204,13.636524,0.382209,0.402408,0.024947,5.422717,19.828626,2.672803,1.249492,0.009769
LOFOTEN-BARENTS SEA,1.205090,0.162887,0.097732,0.060255,0.903818,2.357469,0.003655,0.005070,0.000226,0.077380,0.214856,0.019237,0.010721,0.000126
NORTH SEA,3.734861,0.432007,0.259204,0.186743,2.801145,5.429254,0.174091,0.214789,0.012058,2.258631,9.158609,1.165917,0.670089,0.004399
NORWEGIAN SEA2,2.734769,0.351290,0.210774,0.136738,2.051077,3.530953,0.041571,0.033790,0.001239,0.746265,1.440710,0.130597,0.348258,0.002907
SKAGERAK,6.296219,0.128052,0.076831,0.314811,4.722165,2.318848,0.162892,0.148759,0.011424,2.340441,9.014451,1.357052,0.220424,0.002337


In [8]:
# Update sheet 5a
pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu']

update_spreadsheet_point_sources(temp_path, '5a', pars, 'sew', sew_df)

C:\Data\Anaconda2\lib\site-packages\ipykernel_launcher.py:66: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


### 2.2. Sheet 5b: Industrial effluents

In [9]:
# Get industrial data
cols = [i for i in umon_df.columns if i.split('_')[0]=='ind']
ind_df = umon_df[cols].copy()

# Convert units
ind_df['ind_Hg'] = ind_df['ind_Hg']/1000. # kg to tonnes
ind_df['ind_nh4'] = ind_df['ind_nh4']/1000. # tonnes to ktonnes
ind_df['ind_no3'] = ind_df['ind_no3']/1000. # tonnes to ktonnes
ind_df['ind_n'] = ind_df['ind_n']/1000. # tonnes to ktonnes
ind_df['ind_po4'] = ind_df['ind_po4']/1000. # tonnes to ktonnes
ind_df['ind_p'] = ind_df['ind_p']/1000. # tonnes to ktonnes
ind_df['ind_S.P.M.'] = ind_df['ind_S.P.M.']/1000. # tonnes to ktonnes

ind_df

,ind_n,ind_p,ind_po4,ind_no3,ind_nh4,ind_S.P.M.,ind_TOC,ind_As,ind_Pb,ind_Cd,ind_Cu,ind_Zn,ind_Ni,ind_Cr,ind_Hg
ospar_region,,,,,,,,,,,,,,,
NORWAY,2.471054,0.290061,0.174036,0.123553,1.853291,33.006717,1004.0977,1.948394,1.100930,0.080823,5.144049,15.774354,6.495160,1.486808,0.009491
LOFOTEN-BARENTS SEA,0.081010,0.005899,0.003539,0.004051,0.060758,5.458320,370.7891,0.000141,0.000029,0.000008,0.000572,0.001133,0.000288,0.000069,0.000109
NORTH SEA,0.528998,0.076755,0.046053,0.026450,0.396748,22.980799,466.0137,1.527040,0.731473,0.054724,0.655680,4.555677,4.387587,0.728440,0.002738
NORWEGIAN SEA2,0.910539,0.157877,0.094726,0.045527,0.682904,3.126785,96.8410,0.005302,0.105784,0.011335,0.251153,4.330596,0.113109,0.061616,0.000471
SKAGERAK,0.950508,0.049529,0.029718,0.047525,0.712881,1.440813,70.4539,0.415911,0.263644,0.014757,4.236643,6.886948,1.994176,0.696682,0.006174


In [10]:
# Update sheet 5b
pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu', 'TOC']

update_spreadsheet_point_sources(temp_path, '5b', pars, 'ind', ind_df)

C:\Data\Anaconda2\lib\site-packages\ipykernel_launcher.py:66: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


### 2.3. Sheet 5c: Aquaculture discharges

In [11]:
# Get fish data
cols = [i for i in umon_df.columns if i.split('_')[0]=='fish']
fish_df = umon_df[cols].copy()

# Convert units
fish_df['fish_nh4'] = fish_df['fish_nh4']/1000. # tonnes to ktonnes
fish_df['fish_no3'] = fish_df['fish_no3']/1000. # tonnes to ktonnes
fish_df['fish_n'] = fish_df['fish_n']/1000. # tonnes to ktonnes
fish_df['fish_po4'] = fish_df['fish_po4']/1000. # tonnes to ktonnes
fish_df['fish_p'] = fish_df['fish_p']/1000. # tonnes to ktonnes

fish_df

,fish_n,fish_p,fish_po4,fish_no3,fish_nh4,fish_Cu
ospar_region,,,,,,
NORWAY,59.962278,10.349808,7.141367,6.595851,47.969823,1088.000000
LOFOTEN-BARENTS SEA,15.813914,2.730805,1.884255,1.739531,12.651131,286.488860
NORTH SEA,19.136870,3.299401,2.276586,2.105056,15.309496,346.140265
NORWEGIAN SEA2,24.982000,4.314385,2.976926,2.748020,19.985600,454.823530
SKAGERAK,0.029494,0.005217,0.003600,0.003244,0.023595,0.547345


In [12]:
# Update sheet 5c
pars = ['NH4-N', 'NO3-N', 'P-Total', 'PO4-P', 'N-Total', 'Cu']

update_spreadsheet_point_sources(temp_path, '5c', pars, 'fish', fish_df)

C:\Data\Anaconda2\lib\site-packages\ipykernel_launcher.py:66: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


### 2.4. Sheet 5d: Other discharges

This sheet is left blank

### 2.5. Sheet 5e: Total direct discharges

The sum of sewage, industrial and fish-farm discharges.

In [13]:
# Combine sew, ind and fish, then aggregate
for df in [sew_df, ind_df, fish_df]:
    df.reset_index(inplace=True)
    df.columns = [i.split('_')[1] for i in df.columns]

td_df = pd.concat([sew_df, ind_df, fish_df], axis=0, sort=True)
td_df = td_df.groupby('region').sum()

td_df.columns = ['td_'+i for i in td_df.columns]

td_df

,td_As,td_Cd,td_Cr,td_Cu,td_Hg,td_Ni,td_Pb,td_S.P.M.,td_TOC,td_Zn,td_n,td_nh4,td_no3,td_p,td_po4
region,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,0.003796,0.000234,0.010790,286.566812,0.000235,0.019525,0.005099,7.815789,370.7891,0.215989,17.100014,13.615706,1.803836,2.899591,1.985527
NORTH SEA,1.701131,0.066782,1.398529,349.054577,0.007137,5.553504,0.946262,28.410053,466.0137,13.714286,23.400728,18.507390,2.318249,3.808162,2.581843
NORWAY,2.330603,0.105770,2.736300,1098.566765,0.019260,9.167963,1.503338,46.643241,1004.0977,35.602980,76.404272,60.301318,7.417950,11.714104,7.959945
NORWEGIAN SEA2,0.046873,0.012574,0.409874,455.820948,0.003378,0.243706,0.139574,6.657738,96.8410,5.771306,28.627307,22.719581,2.930285,4.823552,3.282426
SKAGERAK,0.578803,0.026181,0.917106,7.124429,0.008511,3.351228,0.412403,3.759661,70.4539,15.901399,7.276222,5.458641,0.365581,0.182798,0.110149


In [14]:
# Update sheet 5e
pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu', 'TOC']

update_spreadsheet_point_sources(temp_path, '5e', pars, 'td', td_df)

C:\Data\Anaconda2\lib\site-packages\ipykernel_launcher.py:66: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


### 2.6. Sheet 6a: Monitored rivers

**Does "Inner Oslofjord" in the template correspond to "Alna"?** If so, I can fill-in one additional row in this table.

In [15]:
mon_df.reset_index(inplace=True)
tot_df = mon_df.groupby('ospar_region').sum()
trib_df = mon_df[mon_df['old_rid_group']!='rid_11'].groupby('ospar_region').sum()

In [16]:
def update_spreadsheet_monitored_rivers(xlsx, sheet, pars, df_dict):
    """ Update the OSPAR template for monitored rivers.
    
    Args:
        xslx:    Str. Path to Excel template
        sheet:   Str. Sheet name to update
        pars:    List. Parameter names in template to fill-in
        df_dict: Dict. {'tot':tot_df, 'trib':trib_df, 'main':rid11_df}
                 Values to fill-in
        
    Returns:
        None. The template is updated and saved.
    """
    import pandas as pd
    from openpyxl import load_workbook

    # Map Excel headings to df cols
    par_dict = {'SPM':'SPM',
                'TOC':'TOC',
                'PO4-P':'PO4-P',
                'P-Total':'TOTP',
                'NO3-N':'NO3-N', 
                'NH4-N':'NH4-N',
                'N-Total':'TOTN',
                'As':'As',
                'Pb':'Pb',
                'Cd':'Cd', 
                'Cu':'Cu',
                'Zn':'Zn',
                'Ni':'Ni',
                'Total Cr':'Cr',
                'Hg':'Hg'}
  
    # Map template names to df names and rows
    names_dict = {'Norwegian Sea (NO)':('NORWEGIAN SEA2', 'tot'),
                  'Barents Sea (NO)':('LOFOTEN-BARENTS SEA', 'tot'),
                  'Skagerrak (NO)':('SKAGERAK', 'tot'),
                  'North Sea (NO)':('NORTH SEA', 'tot'),
                  'Norway Total':('NORWAY', 'tot'),
                  'Tributary Rivers - Norwegian Sea':('NORWEGIAN SEA2', 'trib'),
                  'Tributary Rivers - Barents Sea':('LOFOTEN-BARENTS SEA', 'trib'),
                  'Tributary Rivers - Skagerak':('SKAGERAK', 'trib'),
                  'Tributary Rivers - North Sea':('NORTH SEA', 'trib'),
                  'Orkla':(29778, 'main'),
                  'Vefsna':(29782, 'main'),
                  'Alta':(29779, 'main'),
                  'Glomma':(29617, 'main'),
                  'Drammenselva':(29612, 'main'),
                  u'Numedalslågen':(29615, 'main'),
                  'Skienselva':(29613, 'main'),
                  'Otra':(29614, 'main'),
                  'Orreelva':(29783, 'main'),
                  'Vosso':(29821, 'main')}

    # Open new file and get sheet
    wb = load_workbook(filename=xlsx)
    ws = wb[sheet]

    # Get row numbers
    row_dict = {}
    for item in ws['B12':'B%s' % ws.max_row]:
        # Get cell properties
        cell = item[0]
        name = cell.value
        row = cell.row
        row_dict[name] = row

    # Get col numbers
    col_dict = {}
    for cell in ws['E9':'AK9'][0]:
        # Get cell properties
        par = cell.value
        col = cell.column
        col_dict[par] = col

    # Update spreadsheet
    for reg in names_dict.keys():
        df_idx, df_name = names_dict[reg]
        
        # Get df 
        df = df_dict[df_name]
        
        for par in pars:
            # Get value from df
            val = df.ix[df_idx, par_dict[par]]

            # Get cell co-ords
            row = row_dict[reg] + 2
            col = col_dict[par]

            # Write value
            ws['%s%s' % (col, row)] = val

    # Save
    wb.save(xlsx)

In [17]:
# Update sheet 6a
pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu', 'TOC']
df_dict = {'tot':tot_df,
           'main':rid11_df,
           'trib':trib_df}

update_spreadsheet_monitored_rivers(temp_path, '6a', pars, df_dict)

C:\Data\Anaconda2\lib\site-packages\ipykernel_launcher.py:85: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


### 2.7. Sheet 6b: Unmonitored areas

In [18]:
# Get diff data
cols = [i for i in umon_df.columns if i.split('_')[0]=='diff']
diff_df = umon_df[cols].copy()

# Convert units
diff_df['diff_nh4'] = diff_df['diff_nh4']/1000. # tonnes to ktonnes
diff_df['diff_no3'] = diff_df['diff_no3']/1000. # tonnes to ktonnes
diff_df['diff_n'] = diff_df['diff_n']/1000. # tonnes to ktonnes
diff_df['diff_po4'] = diff_df['diff_po4']/1000. # tonnes to ktonnes
diff_df['diff_p'] = diff_df['diff_p']/1000. # tonnes to ktonnes

diff_df

,diff_n,diff_p,diff_po4,diff_no3,diff_nh4
ospar_region,,,,,
NORWAY,39.601491,0.771097,0.189690,24.750932,2.178082
LOFOTEN-BARENTS SEA,6.358553,0.124828,0.030708,3.974095,0.349720
NORTH SEA,15.986431,0.252954,0.062227,9.991520,0.879254
NORWEGIAN SEA2,14.197474,0.304884,0.075001,8.873421,0.780861
SKAGERAK,3.059032,0.088430,0.021754,1.911895,0.168247


In [19]:
# Update sheet 6b
pars = ['NH4-N', 'NO3-N', 'P-Total', 'PO4-P', 'N-Total']

update_spreadsheet_point_sources(temp_path, '6b', pars, 'diff', diff_df)

C:\Data\Anaconda2\lib\site-packages\ipykernel_launcher.py:66: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


### 2.8. Sheet 6c: Total inputs

**Note:** See e-mail from Csilla received 07/11/2017 at 13.43. This table should **not** include "point" discharges (`td_df`) - it's just the sum of tables 6a and 6b.

In [20]:
# Rename cols in diff_df
col_map = {'diff_n':'TOTN',
           'diff_p':'TOTP',
           'diff_po4':'PO4-P',
           'diff_no3':'NO3-N',
           'diff_nh4':'NH4-N'}
diff_df.columns = [col_map[i] for i in diff_df.columns]

# Add to total_df
for col in diff_df.columns:
    tot_df[col] = tot_df[col] + diff_df[col]

# Update sheet 6c
pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu', 'TOC']

df_dict = {'tot':tot_df,
           'main':rid11_df,
           'trib':trib_df}

update_spreadsheet_monitored_rivers(temp_path, '6c', pars, df_dict)

C:\Data\Anaconda2\lib\site-packages\ipykernel_launcher.py:85: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


The following two cells are not longer used as I originally misunderstood what table 6c represents.

In [ ]:
## Standardise col names
#diff_df.columns = [i.split('_')[1] for i in diff_df.columns]
#td_df.columns = [i.split('_')[1] for i in td_df.columns]
#td_df.index.name = 'ospar_region'
#
#col_dict = {'SPM':'S.P.M.',
#            'TOTN':'n',
#            'NH4-N':'nh4',
#            'NO3-N':'no3',
#            'TOTP':'p',
#            'PO4-P':'po4'}
#for col in col_dict.keys():
#    new_col = col_dict[col]
#    tot_df[new_col] = tot_df[col]
#    del tot_df[col]
#    
## Reset index
#diff_df.reset_index(inplace=True)
#tot_df.reset_index(inplace=True)
#td_df.reset_index(inplace=True)
#
## Concat and aggregate
##tot_df = pd.concat([diff_df, td_df, tot_df], axis=0).groupby('ospar_region').sum() # See comment above
#tot_df = pd.concat([diff_df, tot_df], axis=0).groupby('ospar_region').sum()
#
## Rename cols
#tot_df.columns = ['tot_'+i for i in tot_df.columns]
#
#tot_df

In [ ]:
## Update sheet 6c
#pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
#        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
#        'N-Total', 'Hg', 'Cu', 'TOC']
#
#update_spreadsheet_point_sources(temp_path, '6c', pars, 'tot', tot_df)

### 2.9. Sheet 7: Concentrations

In [21]:
# Read data
in_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
          r'\Results\Loads_CSVs\concs_and_flows_rid_20_2017.csv')
conc_df = pd.read_csv(in_csv, index_col = 0, encoding='utf-8')

# Convert units
conc_df['Hg_ng/l'] = conc_df['Hg_ng/l'] / 1000 # ng to ug
conc_df[u'NH4-N_µg/l N'] = conc_df[u'NH4-N_µg/l N'] / 1000 # ug to mg
conc_df[u'NO3-N_µg/l N'] = conc_df[u'NO3-N_µg/l N'] / 1000 # ug to mg
conc_df[u'TOTN_µg/l N'] = conc_df[u'TOTN_µg/l N'] / 1000 # ug to mg
conc_df[u'TOTP_µg/l P'] = conc_df[u'TOTP_µg/l P'] / 1000 # ug to mg
conc_df[u'PO4-P_µg/l P'] = conc_df[u'PO4-P_µg/l P'] / 1000 # ug to mg
conc_df['TOC_mg C/l'] = conc_df['TOC_mg C/l'] * 1000 # mg to ug

# Get flags
cols = [i for i in conc_df.columns if i.split('_')[1]=='flag']
lod_df = conc_df[cols]
lod_df.columns = [i.split('_')[0] for i in lod_df.columns]

# Get vals
cols = [i for i in conc_df.columns if ((i.split('_')[0] in lod_df.columns) 
                                       and (i.split('_')[1] != 'flag'))]
conc_df = conc_df[cols]
conc_df.columns = [i.split('_')[0] for i in conc_df.columns]

# Rename
col_dict = {'SPM':'S.P.M.',
            'TOTN':'n',
            'NH4-N':'nh4',
            'NO3-N':'no3',
            'TOTP':'p',
            'PO4-P':'po4'}
for col in col_dict.keys():
    new_col = col_dict[col]
    lod_df[new_col] = lod_df[col]
    conc_df[new_col] = conc_df[col]
    del conc_df[col], lod_df[col]

# Map Excel headings to df cols
par_dict = {'SPM':'S.P.M.',
            'TOC':'TOC',
            'PO4-P':'po4',
            'P-Total':'p',
            'NO3-N':'no3', 
            'NH4-N':'nh4',
            'N-Total':'n',
            'As':'As',
            'Pb':'Pb',
            'Cd':'Cd', 
            'Cu':'Cu',
            'Zn':'Zn',
            'Ni':'Ni',
            'Total Cr':'Cr',
            'Hg':'Hg'}
    
# Map names to stns
names_dict = {'Orkla':29778,
              'Vefsna':29782,
              'Alta':29779,
              'Glomma':29617,
              'Drammenselva':29612,
              u'Numedalslågen':29615,
              'Skienselva':29613,
              'Otra':29614,
              'Orreelva':29783,
              'Vosso':29821}

# Open new file and get sheet
wb = load_workbook(filename=temp_path)
ws = wb['7']

# Get row numbers
row_dict = {}
for item in ws['B12':'B%s' % ws.max_row]:
    # Get cell properties
    cell = item[0]
    name = cell.value
    row = cell.row
    row_dict[name] = row

# Get col numbers
col_dict = {}
for cell in ws['E9':'AK9'][0]:
    # Get cell properties
    par = cell.value
    col = cell.column
    col_dict[par] = col

pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu', 'TOC']

# Update spreadsheet
for reg in names_dict.keys():
    for par in pars:
        # Get values from df
        # 1a. Lower average
        vals = conc_df.ix[names_dict[reg], par_dict[par]].values.copy()
        lods = lod_df.ix[names_dict[reg], par_dict[par]].fillna('0').values.copy()
        vals[(lods=='<')] = 0
        val = np.nanmean(vals)
        
        row = row_dict[reg]
        col = col_dict[par]
        ws['%s%s' % (col, row)] = val
        
        # 1b. Upper average
        val = conc_df.ix[names_dict[reg], par_dict[par]].mean()
        row = row_dict[reg]
        col = col_dict[par]
        ws['%s%s' % (col, row+1)] = val

        # 2. Min
        val = conc_df.ix[names_dict[reg], par_dict[par]].min()
        row = row_dict[reg] + 2
        col = col_dict[par]
        ws['%s%s' % (col, row)] = val

        # 3. Max
        val = conc_df.ix[names_dict[reg], par_dict[par]].max()
        row = row_dict[reg] + 3
        col = col_dict[par]
        ws['%s%s' % (col, row)] = val   
        
        # 4. N
        n_samp = len(conc_df.ix[names_dict[reg], par_dict[par]].dropna())
        row = row_dict[reg] + 5
        col = col_dict[par]
        ws['%s%s' % (col, row)] = n_samp 

        # 5. N LOD
        n_lod = (~pd.isnull(lod_df.ix[names_dict[reg], par_dict[par]])).sum()
        pct_lod = 100 * float(n_lod)/float(n_samp)
        
        if pct_lod < 30:
            val = 'Yes'
        else:
            val = 'No'
            
        row = row_dict[reg] + 4
        col = col_dict[par]
        ws['%s%s' % (col, row)] = val

        # 6. Std
        val = conc_df.ix[names_dict[reg], par_dict[par]].std()
        row = row_dict[reg] + 7
        col = col_dict[par]
        ws['%s%s' % (col, row)] = val
        
# Save
wb.save(temp_path)

C:\Data\Anaconda2\lib\site-packages\ipykernel_launcher.py:98: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Data\Anaconda2\lib\site-packages\ipykernel_launcher.py:99: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Data\Anaconda2\lib\site-packages\ipykernel_launcher.py:108: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Data\Anaconda2\lib\site-packages\ipykernel_launcher.py:114: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based 

### 2.10. Sheet 9: Discharge

The notebook [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/recalculate_ospar_flows.ipynb) has already recalculated the OSPAR flow data. This can simply be copied manually into the final template from here:

C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet\Recalculate_OSPAR_Flows\Updated